# Vogel's approximation
## For assignment1

### Q2 use all capacity

In [1]:
import numpy as np
import pandas as pd

# store the names of plant locations
names=['Mexico','Canada','Chile','Frankfurt','Austin','Japan']

# demand is already known
d=np.array([3,2.6,16,20,26.4,11.9])

# capacity is already known
s=np.array([22,3.7,4.5,47,18.5,5])

# since sum(s) is larger than sum(d), can create a void demander who will import all remaining products
d=np.array([3,2.6,16,20,26.4,11.9,sum(s)-sum(d)])

# build the matrix to be optimized
x=np.zeros((len(s), len(d)))

# create cost matrix
# production cost
c_production=np.array([[92.63,93.25,112.31,73.34,89.15,149.24]])
c_production=np.repeat(c_production, len(d)).reshape(len(s),len(d))

# duties cost, use 0 for the void demander
c_duties=np.array([[0.6,0,0.5,0.095,0.045,0.06,0]])
c_duties=np.repeat(c_duties, len(s)).reshape(len(s),len(d),order="F")
c_duties[range(len(s)),range(len(s))]=0
c_duties=c_duties+1

# shipping cost, use 0 for the void demander
c_shipping=np.array([[0,11.4,7,11,11,14,0],
                     [11,0,9,11.5,6,13,0],
                     [7,10,0,13,10.4,14.3,0],
                     [10,11.5,12.5,0,11.2,13.3,0],
                     [10,6,11,10,0,12.5,0],
                     [14,13,12.5,14.2,13,0,0]])

c=c_production*c_duties+c_shipping

# use a large m to present an infinite large
m=c.max()*1000

# change the cost of the void demander to m
c[:,-1]=m

In [2]:
# start optimaization
while (sum(s)-d[-1])>1e-5:
    # exclude optimized rows and columns
    a=np.repeat(s!=0, len(d)).reshape(c.shape)
    b=np.repeat(d!=0, len(s)).reshape(c.shape,order="F")
    c[(a & b)!=True]=m

    # calculate opportunity cost arrays
    minc_s=np.repeat(c.min(axis=1), c.shape[1]).reshape(c.shape)
    opcost_s=c-minc_s
    opcost_s=np.sort(opcost_s, axis=1)
    opcost_s=opcost_s[:,1]

    minc_d=np.repeat(c.min(axis=0), c.shape[0]).reshape(c.shape,order="F")
    opcost_d=c-minc_d
    opcost_d=np.sort(opcost_d, axis=0)
    opcost_d=opcost_d[1,:]

    # find the index of s and d to be optimized
    maxcost=max(max(opcost_s),max(opcost_d))
    rows=opcost_s==maxcost
    cols=opcost_d==maxcost

    index1=np.repeat(rows, c.shape[1]).reshape(c.shape)
    index2=np.repeat(cols, c.shape[0]).reshape(c.shape,order="F")
    index=index1 | index2
    c_index=c*index
    ctmp=c_index+(c_index==0)*c_index.max() # replace 0 by the max number, therefore can use min() to find the minimum positive

    i=np.where(ctmp==ctmp.min())[0][0]
    j=np.where(ctmp==ctmp.min())[1][0]

    # upgrade x,s,d
    x[i,j]=min(s[i],d[j])
    s[i]=s[i]-x[i,j]
    d[j]=d[j]-x[i,j]

    # output results after each iteration
    print("x=",x)
    print("d=",d)
    print("s=",s)
    print("\n")

x= [[3. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
d= [ 0.   2.6 16.  20.  26.4 11.9 20.8]
s= [19.   3.7  4.5 47.  18.5  5. ]


x= [[ 3.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 20.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
d= [ 0.   2.6 16.   0.  26.4 11.9 20.8]
s= [19.   3.7  4.5 27.  18.5  5. ]


x= [[ 3.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9  0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]]
d= [ 0.   2.6 16.   0.  26.4  0.  20.8]
s= [19.   3.7  4.5 15.1 18.5  5. ]


x= [[ 3.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   4.5  0.   0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9  0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0. 

In [3]:
c=c_production*c_duties+c_shipping
sum(sum(x*c)/100)

74.09026435

### Q3 use 85% of all capacity

In [4]:
import numpy as np
import pandas as pd

# store the names of plant locations
names=['Mexico','Canada','Chile','Frankfurt','Austin','Japan']

# demand is already known
d=np.array([3,2.6,16,20,26.4,11.9])

# capacity is already known
s=np.array([22,3.7,4.5,47,18.5,5])
s=s*0.85

# since sum(s) is larger than sum(d), can create a void demander who will import all remaining products
d=np.array([3,2.6,16,20,26.4,11.9,sum(s)-sum(d)])

# build the matrix to be optimized
x=np.zeros((len(s), len(d)))

# create cost matrix
# production cost
c_production=np.array([[92.63,93.25,112.31,73.34,89.15,149.24]])
c_production=np.repeat(c_production, len(d)).reshape(len(s),len(d))

# duties cost, use 0 for the void demander
c_duties=np.array([[0.6,0,0.5,0.095,0.045,0.06,0]])
c_duties=np.repeat(c_duties, len(s)).reshape(len(s),len(d),order="F")
c_duties[range(len(s)),range(len(s))]=0
c_duties=c_duties+1

# shipping cost, use 0 for the void demander
c_shipping=np.array([[0,11.4,7,11,11,14,0],
                     [11,0,9,11.5,6,13,0],
                     [7,10,0,13,10.4,14.3,0],
                     [10,11.5,12.5,0,11.2,13.3,0],
                     [10,6,11,10,0,12.5,0],
                     [14,13,12.5,14.2,13,0,0]])

c=c_production*c_duties+c_shipping

# use a large m to present an infinite large
m=c.max()*1000

# change the cost of the void demander to m
c[:,-1]=m

In [5]:
# start optimaization
while (sum(s)-d[-1])>1e-5:
    # exclude optimized rows and columns
    a=np.repeat(s!=0, len(d)).reshape(c.shape)
    b=np.repeat(d!=0, len(s)).reshape(c.shape,order="F")
    c[(a & b)!=True]=m

    # calculate opportunity cost arrays
    minc_s=np.repeat(c.min(axis=1), c.shape[1]).reshape(c.shape)
    opcost_s=c-minc_s
    opcost_s=np.sort(opcost_s, axis=1)
    opcost_s=opcost_s[:,1]

    minc_d=np.repeat(c.min(axis=0), c.shape[0]).reshape(c.shape,order="F")
    opcost_d=c-minc_d
    opcost_d=np.sort(opcost_d, axis=0)
    opcost_d=opcost_d[1,:]

    # find the index of s and d to be optimized
    maxcost=max(max(opcost_s),max(opcost_d))
    rows=opcost_s==maxcost
    cols=opcost_d==maxcost

    index1=np.repeat(rows, c.shape[1]).reshape(c.shape)
    index2=np.repeat(cols, c.shape[0]).reshape(c.shape,order="F")
    index=index1 | index2
    c_index=c*index
    ctmp=c_index+(c_index==0)*c_index.max() # replace 0 by the max number, therefore can use min() to find the minimum positive

    i=np.where(ctmp==ctmp.min())[0][0]
    j=np.where(ctmp==ctmp.min())[1][0]

    # upgrade x,s,d
    x[i,j]=min(s[i],d[j])
    s[i]=s[i]-x[i,j]
    d[j]=d[j]-x[i,j]

    # output results after each iteration
    print("x=",x)
    print("d=",d)
    print("s=",s)
    print("\n")

x= [[3. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
d= [ 0.     2.6   16.    20.    26.4   11.9    5.695]
s= [15.7    3.145  3.825 39.95  15.725  4.25 ]


x= [[ 3.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 20.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
d= [ 0.     2.6   16.     0.    26.4   11.9    5.695]
s= [15.7    3.145  3.825 19.95  15.725  4.25 ]


x= [[ 3.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9  0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]]
d= [ 0.     2.6   16.     0.    26.4    0.     5.695]
s= [15.7    3.145  3.825  8.05  15.725  4.25 ]


x= [[ 3.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.  

In [6]:
c=c_production*c_duties+c_shipping
sum(sum(x*c)/100)

78.98513539250001

### Q4 not using Chile, use 90% of all capacity

In [7]:
import numpy as np
import pandas as pd

# store the names of plant locations
names=['Mexico','Canada','Chile','Frankfurt','Austin','Japan']

# demand is already known
d=np.array([3,2.6,16,20,26.4,11.9])

# capacity is already known
s=np.array([22,3.7,0,47,18.5,5])
s=s*0.9

# since sum(s) is larger than sum(d), can create a void demander who will import all remaining products
d=np.array([3,2.6,16,20,26.4,11.9,sum(s)-sum(d)])

# build the matrix to be optimized
x=np.zeros((len(s), len(d)))

# create cost matrix
# production cost
c_production=np.array([[92.63,93.25,112.31,73.34,89.15,149.24]])
c_production=np.repeat(c_production, len(d)).reshape(len(s),len(d))

# duties cost, use 0 for the void demander
c_duties=np.array([[0.6,0,0.5,0.095,0.045,0.06,0]])
c_duties=np.repeat(c_duties, len(s)).reshape(len(s),len(d),order="F")
c_duties[range(len(s)),range(len(s))]=0
c_duties=c_duties+1

# shipping cost, use 0 for the void demander
c_shipping=np.array([[0,11.4,7,11,11,14,0],
                     [11,0,9,11.5,6,13,0],
                     [7,10,0,13,10.4,14.3,0],
                     [10,11.5,12.5,0,11.2,13.3,0],
                     [10,6,11,10,0,12.5,0],
                     [14,13,12.5,14.2,13,0,0]])

c=c_production*c_duties+c_shipping

# use a large m to present an infinite large
m=c.max()*1000

# change the cost of the void demander to m
c[:,-1]=m

In [8]:
# start optimaization
while (sum(s)-d[-1])>1e-5:
    # exclude optimized rows and columns
    a=np.repeat(s!=0, len(d)).reshape(c.shape)
    b=np.repeat(d!=0, len(s)).reshape(c.shape,order="F")
    c[(a & b)!=True]=m

    # calculate opportunity cost arrays
    minc_s=np.repeat(c.min(axis=1), c.shape[1]).reshape(c.shape)
    opcost_s=c-minc_s
    opcost_s=np.sort(opcost_s, axis=1)
    opcost_s=opcost_s[:,1]

    minc_d=np.repeat(c.min(axis=0), c.shape[0]).reshape(c.shape,order="F")
    opcost_d=c-minc_d
    opcost_d=np.sort(opcost_d, axis=0)
    opcost_d=opcost_d[1,:]

    # find the index of s and d to be optimized
    maxcost=max(max(opcost_s),max(opcost_d))
    rows=opcost_s==maxcost
    cols=opcost_d==maxcost

    index1=np.repeat(rows, c.shape[1]).reshape(c.shape)
    index2=np.repeat(cols, c.shape[0]).reshape(c.shape,order="F")
    index=index1 | index2
    c_index=c*index
    ctmp=c_index+(c_index==0)*c_index.max() # replace 0 by the max number, therefore can use min() to find the minimum positive

    i=np.where(ctmp==ctmp.min())[0][0]
    j=np.where(ctmp==ctmp.min())[1][0]

    # upgrade x,s,d
    x[i,j]=min(s[i],d[j])
    s[i]=s[i]-x[i,j]
    d[j]=d[j]-x[i,j]

    # output results after each iteration
    print("x=",x)
    print("d=",d)
    print("s=",s)
    print("\n")

x= [[3. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]]
d= [ 0.    2.6  16.   20.   26.4  11.9   6.68]
s= [16.8   3.33  0.   42.3  16.65  4.5 ]


x= [[ 3.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 20.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
d= [ 0.    2.6  16.    0.   26.4  11.9   6.68]
s= [16.8   3.33  0.   22.3  16.65  4.5 ]


x= [[ 3.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0. 16. 20.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.]]
d= [ 0.    2.6   0.    0.   26.4  11.9   6.68]
s= [16.8   3.33  0.    6.3  16.65  4.5 ]


x= [[ 3.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  16.  20.   0.   6.3  0. ]
 [ 0.   0.   0.   0.   0.   0.   0. ]
 [ 0.   0

In [9]:
c=c_production*c_duties+c_shipping
sum(sum(x*c)/100)

78.480454795

### use the default production plan

In [10]:
import numpy as np
import pandas as pd

# store the names of plant locations
names=['Mexico','Canada','Chile','Frankfurt','Austin','Japan']

# demand is already known
d=np.array([3,2.6,16,20,26.4,11.9])

# capacity is already known
s=np.array([17.2,2.6,4.1,38,14,4])

# build the matrix to be optimized
x=np.zeros((len(s), len(d)))

# create cost matrix
# production cost
c_production=np.array([[92.63,93.25,112.31,73.34,89.15,149.24]])
c_production=np.repeat(c_production, len(d)).reshape(len(s),len(d))

# duties cost
c_duties=np.array([[0.6,0,0.5,0.095,0.045,0.06]])
c_duties=np.repeat(c_duties, len(s)).reshape(len(s),len(d),order="F")
c_duties[range(len(s)),range(len(s))]=0
c_duties=c_duties+1

# shipping cost
c_shipping=np.array([[0,11.4,7,11,11,14],
                     [11,0,9,11.5,6,13],
                     [7,10,0,13,10.4,14.3],
                     [10,11.5,12.5,0,11.2,13.3],
                     [10,6,11,10,0,12.5],
                     [14,13,12.5,14.2,13,0]])

c=c_production*c_duties+c_shipping

# use a large m to present an infinite large
m=c.max()*1000

In [11]:
# start optimaization
while sum(s)>1e-5:
    # exclude optimized rows and columns
    a=np.repeat(s!=0, len(d)).reshape(c.shape)
    b=np.repeat(d!=0, len(s)).reshape(c.shape,order="F")
    c[(a & b)!=True]=m

    # calculate opportunity cost arrays
    minc_s=np.repeat(c.min(axis=1), c.shape[1]).reshape(c.shape)
    opcost_s=c-minc_s
    opcost_s=np.sort(opcost_s, axis=1)
    opcost_s=opcost_s[:,1]

    minc_d=np.repeat(c.min(axis=0), c.shape[0]).reshape(c.shape,order="F")
    opcost_d=c-minc_d
    opcost_d=np.sort(opcost_d, axis=0)
    opcost_d=opcost_d[1,:]

    # find the index of s and d to be optimized
    maxcost=max(max(opcost_s),max(opcost_d))
    rows=opcost_s==maxcost
    cols=opcost_d==maxcost

    index1=np.repeat(rows, c.shape[1]).reshape(c.shape)
    index2=np.repeat(cols, c.shape[0]).reshape(c.shape,order="F")
    index=index1 | index2
    c_index=c*index
    ctmp=c_index+(c_index==0)*c_index.max() # replace 0 by the max number, therefore can use min() to find the minimum positive

    i=np.where(ctmp==ctmp.min())[0][0]
    j=np.where(ctmp==ctmp.min())[1][0]

    # upgrade x,s,d
    x[i,j]=min(s[i],d[j])
    s[i]=s[i]-x[i,j]
    d[j]=d[j]-x[i,j]

    # output results after each iteration
    print("x=",x)
    print("d=",d)
    print("s=",s)
    print("\n")

x= [[3. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]
d= [ 0.   2.6 16.  20.  26.4 11.9]
s= [14.2  2.6  4.1 38.  14.   4. ]


x= [[ 3.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0. 20.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]]
d= [ 0.   2.6 16.   0.  26.4 11.9]
s= [14.2  2.6  4.1 18.  14.   4. ]


x= [[ 3.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]]
d= [ 0.   2.6 16.   0.  26.4  0. ]
s= [14.2  2.6  4.1  6.1 14.   4. ]


x= [[ 3.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   4.1  0.   0.   0. ]
 [ 0.   0.   0.  20.   0.  11.9]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]]
d= [ 0.   2.6 11.9  0.  26.4  0. ]
s= [14.2  2.6  0.   6.1 14.   4. ]


x= [[ 3.   0

In [12]:
c=c_production*c_duties+c_shipping
sum(sum(x*c)/100)

79.542131

### using the same production plan, VAM result is worse than the default plan shown in the reading material